In [1]:
import numpy as np
import mat73
from tensorflow.keras.models import load_model
import scipy.io
import matplotlib.pyplot as plt
from matplotlib import cm

2022-03-13 13:42:47.075062: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-13 13:42:47.075085: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
model_filename = '../Best Neural Nets/eeg_cnn_24.h5'
model = load_model(model_filename,  compile=False)

#### Create simulated data (with AWGN) for evaluation

In [ ]:
###################### single source ######################
from forward import solve_forward
from simulation import Simulation

fwd = solve_forward(num_dipoles='10k')
sim = Simulation(fwd=fwd, source_data=None, eeg_data=None)
snr = 20
eeg_noisy, sources = sim.create_evaluate_dataset(n_samples=5000, snr=snr)

print(eeg_noisy.shape)
# np.save('./eval_sim_data/single_source/{}db/eeg_noisy.npy'.format(snr),eeg_noisy)
# np.save('./eval_sim_data/single_source/{}db/sources.npy'.format(snr),sources)

In [2]:
###################### two sources ######################
from forward import solve_forward
from simulation import Simulation

fwd = solve_forward(num_dipoles='10k')
settings = {  'number_of_sources': 2 }
snr = 20
sim = Simulation(fwd=fwd, settings=settings, noisy_eeg=True,target_snr=(True, snr))
sim.simulate(n_samples=500)

eeg_noisy = sim.eeg_data
sources = sim.source_data
source_centers = sim.source_centers

print(eeg_noisy.shape)
np.save('./eval_sim_data/two_sources/{}db/eeg_noisy.npy'.format(snr),eeg_noisy)
np.save('./eval_sim_data/two_sources/{}db/sources.npy'.format(snr),sources)
np.save('./eval_sim_data/two_sources/{}db/source_centers.npy'.format(snr),source_centers)

Forward problem is solved.
Electrodes: (73, 3)
Dipoles: (10092, 3)
Leadfield: (73, 10092)
Simulate Sources.


100%|██████████| 500/500 [00:01<00:00, 465.49it/s]


Simulate EEG.
Project sources to EEG.
Add noise to eeg


100%|██████████| 500/500 [00:00<00:00, 16850.28it/s]

(73, 500)


#### Eval with noisy sim data 


In [ ]:
# topos were generated with matlab with file create_cnn_input.
sources_path = 'single_source'
snr = 20
eeg_topos = mat73.loadmat('./eval_sim_data/{}/{}db/eeg_topos_noisy.mat'.format(sources,snr))['eeg_topos']
eeg_topos = eeg_topos.transpose(2,0,1)
predicted_sources = model.predict(eeg_topos).T
np.save('./eval_sim_data/{}/{}db/predicted_sources.npy'.format(sources,snr),predicted_sources)

#### Eval with real data

In [ ]:
ms='20'
real_data_file = './real_data/{}ms/eeg_topo_real_{}ms.mat'.format(ms,ms)
topos_eval =  scipy.io.loadmat(real_data_file)['Zi'] 

plt.contourf(topos_eval, cmap=cm.get_cmap('viridis'))
cbar = plt.colorbar()
plt.title('Topography for {} ms'.format(ms.replace('_','.')))
plt.show()

topos_eval= np.expand_dims(topos_eval, axis=0)

In [ ]:
topos_eval = mat73.loadmat('../../Downloads/two_sources/eeg_topos.mat')['eeg_topos']
topos_eval = topos_eval.transpose(2,0,1)

In [ ]:
predicted_sources = model.predict(topos_eval).T

In [ ]:
np.save('../../Downloads/pred_real.npy',predicted_sources)

In [ ]:
np.save('./real_data/{}ms/pred_sources_{}.npy'.format(ms,ms),predicted_sources)